In [1]:
from tqdm import tqdm

import pandas as pd
pd.set_option('mode.chained_assignment',  None)

import pymysql, os

from openai import OpenAI

from sklearn.model_selection import train_test_split

def live_db_conn():
    conn = pymysql.connect(host='host', user='user', password='password',autocommit=True,cursorclass=pymysql.cursors.DictCursor, db = "db")
    return conn

In [2]:
r_food_d = pd.read_excel('./input/food_d.xlsx',index_col=0)

r_food_d = r_food_d[r_food_d['intent_cat'] != '삭제']
r_food_d = r_food_d[~r_food_d['지식베이스'].isnull()]

r_food_d.reset_index(inplace=True)
r_food_d.drop('index',axis=1,inplace=True)
r_food_d.head()

,SPEAKER,MQ,개체명,지식베이스,intent_cat
0,고객,거기 둥지 짬뽕이죠?,둥지 짬뽕,둥지 짬뽕/상호,매장_상품정보
1,고객,김밥 전문집이죠?,"김밥, 전문집",김밥/메뉴,매장_상품정보
2,고객,사천성인가요?,사천성,사천성/상호,매장_상품정보
3,고객,루완이죠?,루완,루완/상호,매장_상품정보
4,고객,월령소바죠?,월령소바,월령소바/상호,매장_상품정보


In [3]:
total_kb_list = []

for rows in r_food_d[~r_food_d['지식베이스'].isnull()]['지식베이스'].tolist():

    kb_list = rows.split(', ')

    kb_list = [kb.split('/')[1] for kb in kb_list] 

    total_kb_list.extend(kb_list)

total_kb_list = set(total_kb_list)

kb_dic = {}
kb_list = ['CVC', '가격', '가짓수', '결제방식', '결제종류', '계절', '계좌번호', '공휴일', '교통', '금액', '기능', '기종', '나이',
 '날짜', '대상', '마감시간', '맛', '맵기', '메뉴', '메뉴정보', '배수', '비율', '사이즈', '상품', '상품권', '상호', '상호명', '색상', '성별', '수량',
 '시간', '시설', '시즌', '식기', '앱', '양식', '예약시간', '예약인원', '예약일', '예약자', '요일', '위치',
 '이벤트', '인원', '인증방식', '자리', '장소', '재료', '좌석', '카드사',
 '카드종류', '캐시백', '쿠폰', '페이먼트', '할인율',
 '행사', '혜택', '화폐', '횟수']

kb_list = ['/'+i for i in kb_list]

for k in kb_list:

    kb_dic[k] = k.replace('/','|')

FADF_KB = r_food_d['지식베이스'].tolist()
x_list = []

for x in FADF_KB:

    if type(x) == float:

        x_list.append(x)

    else:
        
        for k,v in kb_dic.items():

            if k in x:

                x = x.replace(k,v)
            
        x_list.append(x)

r_food_d['지식베이스'] = x_list

In [1]:
# RT API GPT4
%env OPENAI_API_KEY=OPENAI_API_KEY

open_api_key = os.getenv("OPENAI_API_KEY")

client = OpenAI(
    # This is the default and can be omitted
    api_key=os.environ.get('OPENAI_API_KEY'),
)

In [38]:
en_df = pd.DataFrame(columns=r_food_d.columns)
error_idx_list = []
idx = 0

for rows in tqdm(r_food_d.values):

    try:

        sentence = rows[1]
        entity_name = rows[2]
        knowledge_base = rows[3]
        knowledge_base_list = knowledge_base.split(', ')
        real_name_list = [kb.split('|')[0] for kb in knowledge_base_list]
        
        intent_cat = rows[4]

        chat_completion = client.chat.completions.create(
            messages=[
                {"role": "user",
                "content": """
                            1단계 - 이중 따옴표로 구분된 제공된 Sentence를 해당하는 한국 문장을 영어로 번역해줘. 영어가 있다면 그대로 사용해.
                            "Sentence": ""{}"" 

                            2단계 - 이중 따옴표로 구분된 제공된 Knowledge 단어들을 Sentence 번역에 사용된 동일한 영어 단어를 사용해서 번역해줘.
                            Knowledge는 따움표 안에 있는 것은 하나의 단어이므로 하나의 세트로 번역해줘.
                            Knowledge: ""{}"".
                        
                            번역 결과는 "Sentence: , 
                            Knowledge: 'Knowledge1(한국어)->Knowledge1(영어)', 'Knowledge2(한국어)->Knowledge2(영어)'" 형태로 내보내줘.

                            Input 예시는 아래와 같아.
                            Sentence: ""짬뽕은 매운 거예요? 매운 게 고추짬뽕이죠?"",
                            Knowledge: ""'짬뽕', '고추짬뽕'""
                            
                            Output의 형식은 Example을 참고해줘.
                            Example: ""
                            Sentence: 'Is jjamppong spicy? Spicy one is gochujjambbong, right?',
                            Knowledge: '짬뽕->jjamppong', '고추짬뽕->gochujjambbong' 
                            
                            최종적으로 Output 내용만 답변에 넣어줘""
                            """.format(sentence, real_name_list)}
            ],
            model="gpt-3.5-turbo"
            )
        
        answer = chat_completion.choices[0].message.content
        answer = answer.replace('(한국어)','').replace('(영어)','')

        if 'Input' in answer:

            answer = answer.split('Output:\n')
        print(answer)
        translation = answer.split('Sentence: ')[1].split(',\n')[0].strip("'")
        translation = translation.split('\nKnowledge')[0].strip('"')

        translated_kb = answer.split('Knowledge: ')[1].split(', ')
        translated_kb = answer.split('Knowledge: ')[1].replace(']','').replace('[','').strip("'").split(', ')

        kb_dic = {}

        for _ in translated_kb:

            kv = _.split('->')
            key = kv[0].strip("'")
            value = kv[1].strip("'")

            kb_dic[key] = value

        for _ in knowledge_base_list:

            real_name = _.split('|')[0]

            if real_name in ['#이름#','#주소#']:

                continue

            knowledge_base = knowledge_base.replace(real_name, kb_dic[real_name])

        new_row = pd.DataFrame([['고객', translation, entity_name, knowledge_base, intent_cat]], columns=r_food_d.columns)
        en_df = pd.concat([en_df,new_row], axis=0)
        idx += 1

    except:

        error_idx_list.append(idx)
        idx += 1

  0%|          | 1/386 [00:01<08:41,  1.35s/it]

""
Output: 
Sentence: 'Is Onnuri gift certificate available too?',
Knowledge: '온누리 상품권->Onnuri gift certificate'
""


  1%|          | 2/386 [00:02<07:50,  1.22s/it]

Output:
Sentence: 'Can I use a gift card for payment?'
Knowledge: '상품권->gift card'


  1%|          | 3/386 [00:03<08:13,  1.29s/it]

...

Output: 
Sentence: 'How can I register on NAMANE?',
Knowledge: 'NAMANE->NAMANE'


100%|█████████▉| 385/386 [07:43<00:01,  1.13s/it]

"
                            Sentence: 'Please tell me how to recharge Alipay',
                            Knowledge: '알리페이->Alipay'
                            "


100%|██████████| 386/386 [07:45<00:00,  1.21s/it]

""
Sentence: '9491-3000-4735-4320 
 Valid thru 0828 CVC798 
 Refund inquiry',
Knowledge: '9491-3000-4735-4320->9491-3000-4735-4320', '0828->0828', '798->798'
""


In [40]:
en_df['지식베이스'] = en_df['지식베이스'].apply(lambda x:x.replace("'\n""",''))

en_df['지식베이스'] = en_df['지식베이스'].apply(lambda x:x.replace(' ""',''))

en_df['지식베이스'] = en_df['지식베이스'].apply(lambda x:x.replace('"',''))

In [80]:
en_df.to_excel('output/TOTAL_EN.xlsx')